# setup

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
import numpy as np
import os
import glob
import cv2 as cv

2023-05-13 17:10:55.249832: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
base_path = os.path.join("../", "data")

# get image's paths: x_train and x_valid

In [5]:
x_train_file = os.path.join(base_path, "preprocessed_train.txt")
with open(x_train_file) as ptf:
  x_train = ptf.read().splitlines()
ptf.close()

In [6]:
x_train[0]

'preprocessed/train/10000_train.png'

In [7]:
x_valid_file = os.path.join(base_path, "preprocessed_valid.txt")
with open(x_valid_file) as pvf:
  x_valid = pvf.read().splitlines()
pvf.close()

In [8]:
x_valid[0]

'preprocessed/valid/1000_valid.png'

## get image's paths: x_train_gaussian and x_valid_gaussian

In [9]:
x_train_gaussian_file = os.path.join(base_path, "gaussian_train.txt")
with open(x_train_gaussian_file) as xgtf:
  x_gaussian_train = xgtf.read().splitlines()
xgtf.close()

In [10]:
x_gaussian_train[0]

'gaussian_images/train/10000_train_blured.png'

In [11]:
x_valid_gaussian_file = os.path.join(base_path, "gaussian_valid.txt")
with open(x_valid_gaussian_file) as xgvf:
  x_gaussian_valid = xgvf.read().splitlines()
xgvf.close()

In [12]:
x_gaussian_valid[0]

'gaussian_images/valid/1000_valid_blured.png'

In [13]:
print("Dimensiones para imagenes originales:", len(x_train), len(x_valid))
print("Dimensiones para imagenes gaussianas:", len(x_gaussian_train), len(x_gaussian_valid))

Dimensiones para imagenes originales: 25462 2121
Dimensiones para imagenes gaussianas: 25462 2121


In [14]:
x_train = x_train[:15000]
x_valid = x_valid[:500]
x_gaussian_train = x_gaussian_train[:15000]
x_gaussian_valid = x_gaussian_valid[:500]

# Autoencoder

In [15]:
OW, NW = 64, 64
OH, NH = 88, 88

In [16]:
# Crear las listas para las imágenes originales y con ruido
original_images = []
noisy_images = []

# Leer las imágenes originales y con ruido
for or_img, ns_img in zip(x_train, x_gaussian_train):

    # Leer las imágenes en escala de grises
    original_image = cv.imread(os.path.join(base_path, or_img))
    noisy_image = cv.imread(os.path.join(base_path, ns_img))

    oh, ow, _ = original_image.shape
    nh, nw, _ = noisy_image.shape


    original_image = cv.resize(original_image, (OW, OH), interpolation = cv.INTER_AREA)
    noisy_image = cv.resize(noisy_image, (NW, NH), interpolation = cv.INTER_AREA)

    # Normalizar los valores de píxeles en el rango [0, 1]
    original_image = original_image.astype(np.float32) / 255.0
    noisy_image = noisy_image.astype(np.float32) / 255.0

    # Agregar las imágenes a las listas
    original_images.append(original_image)
    noisy_images.append(noisy_image)

# Asegurarse de que las imágenes tengan la forma adecuada
original_images = np.expand_dims(original_images, axis=-1)
noisy_images = np.expand_dims(noisy_images, axis=-1)

In [17]:
# Definir la arquitectura del autoencoder

# Definir la arquitectura del autoencoder
def build_autoencoder():
    # Encoder
    input_img = tf.keras.Input(shape=(88, 64, 3))
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    encoded = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
    x = tf.keras.layers.UpSampling2D((2, 2))(x)
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.UpSampling2D((2, 2))(x)
    decoded = tf.keras.layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    # Autoencoder model
    autoencoder = tf.keras.Model(input_img, decoded)
    return autoencoder

# Construir y compilar el modelo del autoencoder
autoencoder = build_autoencoder()
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 88, 64, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 88, 64, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 44, 32, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 32, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 22, 16, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 22, 16, 64)        36928 

2023-05-13 17:13:56.550666: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Cargar los datos de entrenamiento (ejemplo)
# Aquí debes reemplazar 'X_train' con tus propios datos de entrenamiento
X_train = original_images
Y_train = noisy_images

# Entrenar el autoencoder
autoencoder.fit(Y_train, X_train, epochs=10, batch_size=16)

# Realizar predicciones en nuevos datos
# Aquí debes reemplazar 'X_test' con tus propios datos de prueba
#X_test = np.random.rand(10, 128, 100, 3)
#predictions = autoencoder.predict(X_test)

Epoch 1/10


ValueError: in user code:

    File "/Users/brandon/opt/anaconda3/envs/deep_learning/lib/python3.9/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/Users/brandon/opt/anaconda3/envs/deep_learning/lib/python3.9/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/brandon/opt/anaconda3/envs/deep_learning/lib/python3.9/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/Users/brandon/opt/anaconda3/envs/deep_learning/lib/python3.9/site-packages/keras/engine/training.py", line 998, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/Users/brandon/opt/anaconda3/envs/deep_learning/lib/python3.9/site-packages/keras/engine/training.py", line 1092, in compute_metrics
        self.compiled_metrics.update_sta